# Necessary Imports and Declarations

In [2]:
pip install gradio

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.1 MB 4.9 MB/s 
     |████████████████████████████████| 140 kB 47.2 MB/s 
     |████████████████████████████████| 84 kB 3.2 MB/s 
     |████████████████████████████████| 1.1 MB 49.5 MB/s 
     |████████████████████████████████| 212 kB 44.6 MB/s 
     |████████████████████████████████| 84 kB 3.2 MB/s 
     |████████████████████████████████| 57 kB 3.7 MB/s 
     |████████████████████████████████| 272 kB 57.9 MB/s 
     |████████████████████████████████| 54 kB 2.1 MB/s 
     |████████████████████████████████| 2.3 MB 50.0 MB/s 
     |████████████████████████████████| 271 kB 44.4 MB/s 
     |████████████████████████████████| 94 kB 3.4 MB/s 
     |████████████████████████████████| 144 kB 58.7 MB/s 
     |████████████████████████████████| 63 kB 1.6 MB/s 
     |████████████████████████████████| 80 kB 7.9 MB/s 
     |████████████████████████████████| 68 kB 

In [3]:
!wget https://github.com/Jugarcia15/Google-Colab-Retrieval-Repo/raw/main/MN_Full_model.h5

--2022-07-08 23:20:16--  https://github.com/Jugarcia15/Google-Colab-Retrieval-Repo/raw/main/MN_Full_model.h5
Resolving github.com (github.com)... 140.82.112.3
Connecting to github.com (github.com)|140.82.112.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/Jugarcia15/Google-Colab-Retrieval-Repo/main/MN_Full_model.h5 [following]
--2022-07-08 23:20:17--  https://raw.githubusercontent.com/Jugarcia15/Google-Colab-Retrieval-Repo/main/MN_Full_model.h5
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10495528 (10M) [application/octet-stream]
Saving to: ‘MN_Full_model.h5’

MN_Full_model.h5    100%[===================>]  10.01M  --.-KB/s    in 0.1s    

2022-07-08 23:20:17 (96.4 MB/s) - ‘MN_Full_model.h

In [4]:
!wget https://github.com/Jugarcia15/Google-Colab-Retrieval-Repo/raw/main/test.zip
!unzip '/content/test.zip'

--2022-07-08 23:20:17--  https://github.com/Jugarcia15/Google-Colab-Retrieval-Repo/raw/main/test.zip
Resolving github.com (github.com)... 140.82.113.3
Connecting to github.com (github.com)|140.82.113.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/Jugarcia15/Google-Colab-Retrieval-Repo/main/test.zip [following]
--2022-07-08 23:20:17--  https://raw.githubusercontent.com/Jugarcia15/Google-Colab-Retrieval-Repo/main/test.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1840533 (1.8M) [application/zip]
Saving to: ‘test.zip’

test.zip            100%[===================>]   1.75M  --.-KB/s    in 0.06s   

2022-07-08 23:20:17 (30.4 MB/s) - ‘test.zip’ saved [1840533/1840533]

Archive:  /content/te

In [5]:
import tensorflow as tf
import tensorflow_hub as hub
import gradio as gr
import ipywidgets as widgets
from ipywidgets import interact, interact_manual
from IPython.display import Image
from tensorflow.keras.applications.mobilenet_v3 import preprocess_input
import os
tf.__version__

'2.8.2'

## New Save and load models, Predict functions

In [6]:
def save_model(model, suffix=None):
  modeldir = os.path.join("/content/",
                          datetime.datetime.now().strftime("%Y%m%d-%H%M%s"))
  model_path=modeldir+"-" + suffix + ".h5"
  print(f"Saving model to: {model_path}")
  model.save(model_path)
  return model_path

#create a function to save a model
#from tensorflow.keras.models import load_model
#model.save('1010_images_model_inceptionv3.h5')
def load_model(model_path):
  print(f"loading model from: {model_path}")
  model=tf.keras.models.load_model(model_path,
                                   custom_objects={"KerasLayer":hub.KerasLayer})
  return model

class_names=['Amanita Bisporigera',
                 'Amanita Muscaria',
                 'Boletus Edulis',
                 'Cantharellus',
                 'Russula Mariae'
                 ]

gradio_path = '/content/MN_Full_model.h5'
gradio_model = load_model(gradio_path)

def predict_image(img):
  img = img.reshape((-1,224,224,3))
  img = tf.keras.applications.mobilenet_v3.preprocess_input(img)
  prediction= gradio_model.predict(img).flatten()
  confidences={class_names[i]: float(prediction[i]) for i in range(5)}
  return confidences

loading model from: /content/MN_Full_model.h5


# Gradio App

In [7]:
print('View files in /Content/Test/')
print('These are test images you can download and upload to the fileLoader in Gradio below.')
fdir = '/content/test/'
@interact 
def show_test_images(file=os.listdir('/content/test/')):
  display(Image(fdir+file))

View files in /Content/Test/
These are test images you can download and upload to the fileLoader in Gradio below.


interactive(children=(Dropdown(description='file', options=('ab.jpg', 'ca4.jpg', 'ab1.jpg', 'russula mariae.jp…

In [8]:
start = gr.Interface(
                     title='Mushroom Image Classifier',
                     description='Identifying the species of mushrooms given an image of a mushroom.',
                     fn=predict_image, 
                     inputs = gr.inputs.Image(shape=(224,224)), 
                     outputs = gr.outputs.Label(num_top_classes=5),
                     #interpretation='default', 
                     capture_session=True
                     )

start.launch(debug='True', share =True)

/usr/local/lib/python3.7/dist-packages/gradio/deprecation.py:40: UserWarning: `optional` parameter is deprecated, and it has no effect
  warnings.warn(value)
/usr/local/lib/python3.7/dist-packages/gradio/deprecation.py:40: UserWarning: The 'type' parameter has been deprecated. Use the Number component instead.
  warnings.warn(value)
/usr/local/lib/python3.7/dist-packages/gradio/deprecation.py:40: UserWarning: `capture_session` parameter is deprecated, and it has no effect
  warnings.warn(value)


Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://33672.gradio.app

This share link expires in 72 hours. For free permanent hosting, check out Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.


(<gradio.routes.App at 0x7f59fcc03e90>,
 'http://127.0.0.1:7860/',
 'https://33672.gradio.app')

# Streamlit